# Working with Audio data

In [ ]:
!pip install librosa

In [ ]:
!pip install datasets[audio]

In [ ]:
!pip install transformers

In [ ]:
import librosa

array, sampling_rate = librosa.load(librosa.ex("trumpet"))

In [ ]:
import matplotlib.pyplot as plt
import librosa.display

plt.figure().set_figwidth(12)
librosa.display.waveshow(array, sr=sampling_rate)

In [ ]:
import numpy as np

dft_input = array[:4096]

# calculate the DFT
window = np.hanning(len(dft_input))
windowed_input = dft_input * window
dft = np.fft.rfft(windowed_input)

# get the amplitude spectrum in decibels
amplitude = np.abs(dft)
amplitude_db = librosa.amplitude_to_db(amplitude, ref=np.max)

# get the frequency bins
frequency = librosa.fft_frequencies(sr=sampling_rate, n_fft=len(dft_input))

plt.figure().set_figwidth(12)
plt.plot(frequency, amplitude_db)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude (dB)")
plt.xscale("log")

In [ ]:
import numpy as np

D = librosa.stft(array)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

plt.figure().set_figwidth(12)
librosa.display.specshow(S_db, x_axis="time", y_axis="hz")
plt.colorbar()

In [ ]:
S = librosa.feature.melspectrogram(y=array, sr=sampling_rate, n_mels=128, fmax=8000)
S_dB = librosa.power_to_db(S, ref=np.max)

plt.figure().set_figwidth(12)
librosa.display.specshow(S_dB, x_axis="time", y_axis="mel", sr=sampling_rate, fmax=8000)
plt.colorbar()

In [ ]:
from datasets import load_dataset

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")
minds

In [ ]:
example = minds[0]
example

In [ ]:
id2label = minds.features["intent_class"].int2str
id2label(example["intent_class"])

In [ ]:
columns_to_remove = ["lang_id", "english_transcription"]
minds = minds.remove_columns(columns_to_remove)
minds

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr


def generate_audio():
    example = minds.shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label(example["intent_class"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

In [ ]:
import librosa
import matplotlib.pyplot as plt
import librosa.display

array = example["audio"]["array"]
sampling_rate = example["audio"]["sampling_rate"]

plt.figure().set_figwidth(12)
librosa.display.waveshow(array, sr=sampling_rate)

In [ ]:
from datasets import Audio

minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
minds[0]

In [ ]:
MAX_DURATION_IN_SECONDS = 20.0


def is_audio_length_in_range(input_length):
    return input_length < MAX_DURATION_IN_SECONDS

In [ ]:
# use librosa to get example's duration from the audio file
new_column = [librosa.get_duration(filename=x) for x in minds["path"]]
minds = minds.add_column("duration", new_column)

# use 🤗 Datasets' `filter` method to apply the filtering function
minds = minds.filter(is_audio_length_in_range, input_columns=["duration"])

# remove the temporary helper column
minds = minds.remove_columns(["duration"])
minds

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
def prepare_dataset(example):
    audio = example["audio"]
    features = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"], padding=True
    )
    return features

In [ ]:
audio = minds[0]['audio']
features = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"], padding=True
    )

In [ ]:
features

In [ ]:
minds = minds.map(prepare_dataset)
minds

In [ ]:
minds[0]

In [ ]:
import numpy as np

example = minds[0]
input_features = example["input_features"]

plt.figure().set_figwidth(12)
librosa.display.specshow(
    np.asarray(input_features[0]),
    x_axis="time",
    y_axis="mel",
    sr=feature_extractor.sampling_rate,
    hop_length=feature_extractor.hop_length,
)
plt.colorbar()

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("openai/whisper-small")

# Audio classification with a pipeline

In [ ]:
from datasets import load_dataset
from datasets import Audio

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

In [ ]:
example = minds[0]

In [ ]:
classifier(example["audio"]["array"])

In [ ]:
id2label = minds.features["intent_class"].int2str
id2label(example["intent_class"])

# Automatic speech recognition with a pipeline

In [ ]:
from transformers import pipeline

asr = pipeline("automatic-speech-recognition")

In [ ]:
example = minds[0]
asr(example["audio"]["array"])

In [ ]:
example["english_transcription"]

In [ ]:
from datasets import load_dataset
from datasets import Audio

minds = load_dataset("PolyAI/minds14", name="de-DE", split="train")
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
example = minds[0]
example["transcription"]

In [ ]:
from transformers import pipeline

asr = pipeline("automatic-speech-recognition", model="maxidl/wav2vec2-large-xlsr-german")
asr(example["audio"]["array"])

# Pre-trained models and datasets for audio classification

In [ ]:
pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 40.9 MB/s eta 0:00:00


## Keyword Spotting

In [ ]:
from datasets import load_dataset

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

In [ ]:
classifier(minds[0]["path"])

## Speech Commands

In [ ]:
speech_commands = load_dataset(
    "speech_commands", "v0.02", split="validation", streaming=True
)

In [ ]:
sample = next(iter(speech_commands))

In [ ]:
sample

In [ ]:
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2"
)
classifier(sample["audio"])

## Language Identification

In [ ]:
fleurs = load_dataset("google/fleurs", "all", split="validation", streaming=True)
sample = next(iter(fleurs))

In [ ]:
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
classifier = pipeline(
    "audio-classification", model="sanchit-gandhi/whisper-medium-fleurs-lang-id"
)

In [ ]:
classifier(sample["audio"])

## Zero-Shot Audio Classification

In [ ]:
dataset = load_dataset("ashraq/esc50", split="train", streaming=True)
audio_sample = next(iter(dataset))["audio"]["array"]

In [ ]:
candidate_labels = ["Sound of a dog", "Sound of vacuum cleaner"]

In [ ]:
classifier = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
classifier(audio_sample, candidate_labels=candidate_labels)

In [ ]:
audio_sample

In [ ]:
Audio(audio_sample, rate=16000)

# Fine-tuning a model for music classification

## The Dataset

In [1]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Found cached dataset gtzan (/home/ubuntu/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [2]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

Loading cached split indices for dataset at /home/ubuntu/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-52d2398c8e4ac745.arrow and /home/ubuntu/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-3bcc56e346e4d81c.arrow


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [3]:
gtzan["train"][0]

{'file': '/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/e57ca3048d51a873c49bc5a02b55d4a02879bb90cd1c96d4b0e25eea7eab969f/genres/pop/pop.00098.wav',
 'audio': {'path': '/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/e57ca3048d51a873c49bc5a02b55d4a02879bb90cd1c96d4b0e25eea7eab969f/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

In [4]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

In [5]:
#!pip install gradio

In [6]:
# import gradio as gr

In [7]:
# def generate_audio():
#     example = gtzan["train"].shuffle()[0]
#     audio = example["audio"]
#     return (
#         audio["sampling_rate"],
#         audio["array"],
#     ), id2label_fn(example["genre"])


# with gr.Blocks() as demo:
#     with gr.Column():
#         for _ in range(4):
#             audio, label = generate_audio()
#             output = gr.Audio(audio, label=label)

# demo.launch(debug=True)

## Preprocessing the data

In [25]:
from transformers import AutoFeatureExtractor

model_id = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True #, return_attention_mask=True
)

In [26]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [27]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [28]:
gtzan["train"][0]

{'file': '/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/e57ca3048d51a873c49bc5a02b55d4a02879bb90cd1c96d4b0e25eea7eab969f/genres/pop/pop.00098.wav',
 'audio': {'path': '/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/e57ca3048d51a873c49bc5a02b55d4a02879bb90cd1c96d4b0e25eea7eab969f/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

In [29]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


In [30]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values']
Mean: 0.305, Variance: 0.124


In [31]:
max_duration = 30


def preprocess_function(examples):
    print(len(examples['audio']))
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        # return_attention_mask=True,
    )
    return inputs

In [32]:
import gc
gc.collect()

1011

In [33]:
gtzan_encoded = gtzan.map(
    preprocess_function, remove_columns=["audio", "file"], batched=True, num_proc=1
)
gtzan_encoded

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

899


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

100


DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values'],
        num_rows: 100
    })
})

In [34]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [35]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

In [36]:
from transformers import AutoModelForAudioClassification, ASTForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
     ignore_mismatched_sizes=True
)

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# !pip install accelerate

In [37]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 2
gradient_accumulation_steps = 4
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

In [18]:
# !pip install evaluate

In [38]:
import evaluate
import numpy as np
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [20]:
# !git lfs install

In [39]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
   
)

trainer.train()

/home/ubuntu/kj_learning/Audio-AI/music_classification/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan is already a clone of https://huggingface.co/karanjakhar/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan. Make sure you pull the latest changes with `repo.git_pull()`.
/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,1.069300,0.624481,0.830000
2,0.608400,0.531974,0.810000
2,0.539400,0.468272,0.850000
4,0.157500,0.577224,0.870000
4,0.004900,0.479551,0.880000
6,0.001400,0.420180,0.940000
6,0.000200,0.479560,0.900000
8,0.000200,0.353428,0.890000
8,0.136700,0.373395,0.910000
9,0.000100,0.363943,0.920000


TrainOutput(global_step=1120, training_loss=0.21337628019973637, metrics={'train_runtime': 1783.9762, 'train_samples_per_second': 5.039, 'train_steps_per_second': 0.628, 'total_flos': 6.067675059899597e+17, 'train_loss': 0.21337628019973637, 'epoch': 9.96})

In [40]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [41]:
trainer.push_to_hub(**kwargs)

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/karanjakhar/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan
   7972f02..ac4733b  main -> main

To https://huggingface.co/karanjakhar/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan
   ac4733b..9be9b5d  main -> main



'https://huggingface.co/karanjakhar/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan/commit/ac4733b80f10a7ca94773307a828eec61aa341df'